In [ ]:
import os
import pandas as pd
import geopandas as gpd
import pygeos as pg
import numpy as np
import tensorflow as tf
import sqlalchemy as sq
import calendar
from dotenv import load_dotenv
from IPython.display import clear_output
from matplotlib import pyplot as plt
from DataService import DataService

In [ ]:
# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)
os.chdir('/tf')
PGUSER = os.getenv('POSTGRES_USER')
PGPW = os.getenv('POSTGRES_PW')
PGDB = os.getenv('POSTGRES_DB')
NULLFLAG = -9999

In [ ]:
# Connect to the database
pullService = DataService(PGDB, PGUSER, PGPW)
db_pull_con = pullService.connect()

pushService = DataService(PGDB, PGUSER, PGPW)
db_push_con = pushService.connect()

In [ ]:
def date_iter(year, month):
    """Iterate over days in a month."""
    for i in range(1, calendar.monthrange(year, month)[1] + 1):
        yield i

In [ ]:
def summary(df: pd.DataFrame) -> pd.DataFrame:
    """Returns new df with min max mean for each numeric column"""
    result = df.copy()
    result.groupby('ClimateID', 'ProvinceCode', 'Year', 'Month', 'Day').agg(MeanTemp=('Temp', 'mean'), MinTemp=('Temp', 'min'), MaxTemp=('Temp', 'max'),
                                                                          MeanDewPoint=('Dew', 'mean'), MinDewPoint=('Dew', 'min'), MaxDewPoint=('Dew', 'max'),
                                                                            MeanHumidity=('Humidity', 'mean'), MinHumidity=('Humidity', 'min'), MaxHumidity=('Humidity', 'max'),
                                                                            MeanPressure=('Pressure', 'mean'), MinPressure=('Pressure', 'min'), MaxPressure=('Pressure', 'max'),
                                                                            MeanWindSpeed=('WindSpeed', 'mean'), MinWindSpeed=('WindSpeed', 'min'), MaxWindSpeed=('WindSpeed', 'max'),
                                                                            MeanWindChill=('WindChill', 'mean'), MinWindChill=('WindChill', 'min'), MaxWindChill=('WindChill', 'max'),
                                                                            TotalPrecip=('Precip', 'sum'), MeanWindDirection=('WindDirection', 'mean'))
    return result


In [ ]:
def cleanup(id: str, year: int, month: int, day: int, srcTable: str, destTable: str) -> None:
    # Get the data for the day
    query = "SELECT * FROM public.\"{}\" WHERE \"ClimateID\" like '{}' AND \"Year\" = {} AND \"Month\" = {} AND \"Day\" = {};".format(srcTable, id, year, month, day)
    df = pd.read_sql(query, db_pull_con)
    if df.empty:
        return

    df[['ClimateID', 'ProvinceCode']] = df[['ClimateID', 'ProvinceCode']].astype(str)
    df[['Year', 'Month', 'Day', 'Hour']] = df[['Year', 'Month', 'Day', 'Hour']].astype(int)

    # use float for numeric columns
    df[['Temp', 'DewPointTemp', 'PrecipAmount', 'RelativeHumidity', 'StationPressure', 'WindChill', 'WindDirection', 'WindSpeed']] = df[['Temp', 
                'DewPointTemp', 'PrecipAmount', 'RelativeHumidity', 'StationPressure', 'WindChill', 'WindDirection', 'WindSpeed']].astype(float)

    # Replace NULLFLAG values with mean for each column
    df = df.replace(NULLFLAG, np.nan)
    df = df.fillna(df.mean())

    # Get df with min max mean for each numeric column
    dfSummary = summary(df)

    # Update the database
    dfSummary.to_sql(destTable, db_push_con, if_exists='append', index=False)


In [ ]:
# Load the data
climateIdTable = "TenYrStationsHourly"

# first get distinct list of stations
query = "SELECT DISTINCT \"ClimateID\" FROM public.\"{}\";".format(climateIdTable)
dfIDs = pd.read_sql(query, db_pull_con)


In [ ]:
srcTable = "WeatherDataHourly"
destTable = "WeatherDataHourlyAggDaily"
# Replace NULLFLAG values with mean for each column
for id in dfIDs['ClimateID']:
    clear_output(wait=False)
    print("Processing ClimateID: {}".format(id))

    # Iterate through days from 2009 to 2022
    year = 2010
    month = 1
    for year in range(2010, 2022):
        # first we check if the db has data for the year
        query = "SELECT * FROM public.\"{}\" WHERE \"ClimateID\" like '{}' AND \"Year\" = {};".format(srcTable, id, year)
        dfYear = pd.read_sql(query, db_pull_con)
        if dfYear.empty:
            continue
        for month in range(1, 13):
            for day in date_iter(year, month):
                cleanup(id, year, month, day, srcTable, destTable)
            print("Processed ClimateID: {} for {}-{}".format(id, year, month))